In [1]:
!whoami

bentoml


In [2]:
!ls

anomaly.ipynb	   final-model.pt  requirements.txt
anomaly-new.ipynb  __pycache__	   service.py


# BentoML Scikit-learn Tutorial


This is a sample project demonstrating basic usage of [BentoML](https://github.com/bentoml) with
Scikit-learn.

In this project, we will train a classifier model using Scikit-learn and the Iris dataset, build
a prediction service for serving the trained model via an HTTP server, and containerize the 
model server as a docker image for production deployment.


Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/quickstart

### Install Dependencies

Install required python packages:

In [ ]:
!pip install -r requirements.txt

##  Model Training

In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import classification_report
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [4]:
import glob

In [5]:
class GATLayer(nn.Module):
    
    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensionality of input features
            c_out - Dimensionality of output features
            num_heads - Number of heads, i.e. attention mechanisms to apply in parallel. The 
                        output features are equally split up over the heads if concat_heads=True.
            concat_heads - If True, the output of the different heads is concatenated instead of averaged.
            alpha - Negative slope of the LeakyReLU activation.
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Number of output features must be a multiple of the count of heads."
            c_out = c_out // num_heads
        
        # Sub-modules and parameters needed in the layer
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out)) # One per head
        self.leakyrelu = nn.LeakyReLU(alpha)
        
        # Initialization from the original implementation
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Input features of the node. Shape: [batch_size, c_in]
            adj_matrix - Adjacency matrix including self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - If True, the attention weights are printed during the forward pass (for debugging purposes)
        """
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)
        
        # Apply linear layer and sort nodes by head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)
        # We need to calculate the attention logits for every edge in the adjacency matrix 
        # Doing this on all possible combinations of nodes is very expensive
        # => Create a tensor of [W*h_i||W*h_j] with i and j being the indices of all edges
        edges = adj_matrix.nonzero(as_tuple=False) # Returns indices where the adjacency matrix is not 0 => edges
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # Index select returns a tensor with node_feats_flat being indexed at the desired positions along dim=0
        # Calculate attention MLP output (independent for each head)
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a) 
        attn_logits = self.leakyrelu(attn_logits)
        # Map list of attention values back into a matrix
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)
        # Weighted average of attention
        self.attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", self.attn_probs.permute(0, 3, 1, 2))
        node_feats = torch.einsum('bijh,bjhc->bihc', self.attn_probs, node_feats)
        # If heads should be concatenated, we can do this by reshaping. Otherwise, take mean
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)
        return node_feats 

In [6]:
class ReconstructionModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size,alpha=0.2,dropout_prob=0.3):
        super(ReconstructionModel, self).__init__()


        self.fc1 = nn.Linear(input_size, 192)
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc1_ = nn.Linear(192, hidden_size1)
        self.dropout1_ = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.dropout2 = nn.Dropout(p=dropout_prob)
        self.fc3 = nn.Linear(hidden_size2, hidden_size1)
        self.dropout3 = nn.Dropout(p=dropout_prob)
        self.fc3_ = nn.Linear(hidden_size1, 192)
        self.dropout3_ = nn.Dropout(p=dropout_prob)
        self.fc4 = nn.Linear(192, output_size)

        self.leakyrelu = nn.ReLU()

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.dropout1(x)
        x = self.leakyrelu(self.fc1_(x))
        x = self.dropout1_(x)
        x = self.leakyrelu(self.fc2(x))
        x = self.dropout2(x)
        encoder_out = x
        x = self.leakyrelu(self.fc3(x))
        x = self.dropout3(x)
        x = self.leakyrelu(self.fc3_(x))
        x = self.dropout3_(x)
        # x = torch.sigmoid(self.fc4(x))
        x = self.fc4(x)
        
        return x,encoder_out

In [7]:
class gat_lstm_autoencoder(nn.Module):
  def __init__(self, num_nodes,input_features_size, gat_out_size, lstm_out_size, linear_out_size, reconstruct_hidden_size1, reconstruct_hidden_size2, num_layers, num_heads,batch_size,period):

    super(gat_lstm_autoencoder, self).__init__()


    self.num_nodes = num_nodes
    self.hidden_size = gat_out_size
    self.hidden_size1 = lstm_out_size
    self.input_features_size = input_features_size
    self.batch_size = batch_size
    self.period = period
    self.gat = GATLayer(input_features_size, 16, num_heads)
    self.gat1 = GATLayer(16, 12, num_heads)
    # self.gat2 = GATLayer(16, 12, num_heads)

    self.lstm = nn.LSTM(num_nodes*12, 12*12, 2, batch_first=False, dropout=0.25) #24
    self.lstm1 = nn.LSTM(num_nodes*12, 12*12, 2, batch_first=False, dropout=0.25) #12
    # self.lstm2 = nn.LSTM(num_nodes*8, 12*8, 2, batch_first=False, dropout=0.25) #6
    # self.lstm3 = nn.LSTM(num_nodes*8, 12*8, 2, batch_first=False, dropout=0.25) #12
    # self.lstm4 = nn.LSTM(num_nodes*8, 12*8, 2, batch_first=False, dropout=0.25) #24

    self.gat3 = GATLayer(12, 16, num_heads)
    self.gat4 = GATLayer(16, 24, num_heads)
    self.fc_layers = nn.Sequential(
        nn.Linear(12*24, 12*input_features_size),
        nn.LeakyReLU()
    )

    self.linear = nn.Linear( 12*24, 12*input_features_size)
    # self.linear1 = nn.Linear( 12*16, 12*input_features_size)
    # self.linear2 = nn.Linear( 12*24, 12*20)
    # self.linear2 = nn.Linear( 12*10, 12*20)

    self.dropout = nn.Dropout(p=0.3)
    self.dropout1 = nn.Dropout(p=0.3)
    self.dropout2 = nn.Dropout(p=0.3)
    # self.dropout3 = nn.Dropout(p=0.3)

    self.leakyrelu = nn.ReLU(0.2)

    self.norm = nn.LayerNorm(12*8) 
    self.norm1 = nn.LayerNorm(12*16)
    self.norm2 = nn.LayerNorm(12*input_features_size) 
    self.adj = torch.tensor([#['payment 0', 'shipping 1', 'redis 2', 'mongodb 3', 'dispatch 4', 'rabbitmq 5', 'user 6', 'mysql 7', 'catalogue 8', 'ratings 9', 'web 10', 'cart 11']
       [[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]]])
    
    # self.norm3 = nn.LayerNorm(12*10)
    # self.norm4 = nn.LayerNorm(12*15)
    # self.norm5 = nn.LayerNorm(12*20)
    # self.reconstruct = ReconstructionModel(linear_out_size,reconstruct_hidden_size1, reconstruct_hidden_size2,num_nodes*input_features_size)

  # GAT:    [batch_size,num_nodes,features],[num_nodes,num_nodes]   |   [batch_size,num_nodes,hidden_size]
  # LSTM:   [batch_size,1,num_nodes*hidden_size]                    |   [batch_size,1,num_nodes*hidden_size1]
  # Linear: [batch_size,num_nodes*hidden_size1]                     |   [batch_size,num_nodes*hidden_size2]
  # Recons: [batch_size,num_nodes*hidden_size2]                     |   [batch_size,num_nodes*input_size]
 
  
    
  def forward(self, x):
    INP = x
    batch_size = x.size(0)
    # print('before_gat',x.shape)
    x = self.gat(x, self.adj)
    x = F.elu(x)
    # x = self.leakyrelu(x)
    x = self.gat1(x, self.adj)
    x = F.elu(x)
    # x = self.leakyrelu(x)
    # x = self.gat2(x, self.adj)
    # # x = F.elu(x)
    # x = self.leakyrelu(x)
    gat_out = x

    # print('after_gat',x.shape)
    #x = x.reshape(self.period,batch_size//self.period,self.num_nodes*12)
    x = x.reshape(self.period,-1,self.num_nodes*12)
    x = self.dropout(x)
    # print('before_lstm',x.shape)
    x,(hn,cn) = self.lstm(x)
    decoder_hidden = (hn, torch.zeros_like(hn))
    decoder_input = torch.zeros_like(x)
    encoder_out = x
    x,(hn,cn) = self.lstm1(decoder_input,decoder_hidden)
    
    x = x.reshape(-1,self.num_nodes,12)
    x = self.gat3(x, self.adj)
    x = F.elu(x)
    # x = self.leakyrelu(x)
    x = self.gat4(x, self.adj)
    x = F.elu(x)
    # x = self.leakyrelu(x)

    # x,(hn,cn) = self.lstm2(x[-6:])
    
    # x,(hn,cn) = self.lstm3(torch.cat((x,x),0))
    # x,(hn,cn) = self.lstm4(torch.cat((x,x),0))
    # x = self.norm(x)
    
    
    
    
    x = x.reshape(batch_size,self.num_nodes*24)
    #x = self.linear(x)
    #x = self.leakyrelu(x)
    x = self.fc_layers(x)
    # x = self.dropout1(x)
    
    # x = self.norm1(x)
    
    # x = self.linear1(x)
    # x = self.leakyrelu(x)
    # # x = self.dropout2(x)
    # x = self.norm2(x)

    x_ = x.reshape(batch_size,self.num_nodes,-1)
    out = torch.stack([F.mse_loss(INP[i], x_[i],reduce = 'sum') for i in range(x.shape[0])])
    out = out.reshape([-1,1])
    # x_ = x
    return out#x_,x,gat_out,encoder_out

In [8]:
adj_matrix = torch.Tensor([#['payment 0', 'shipping 1', 'redis 2', 'mongodb 3', 'dispatch 4', 'rabbitmq 5', 'user 6', 'mysql 7', 'catalogue 8', 'ratings 9', 'web 10', 'cart 11']
       [[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]]])

In [9]:
import bentoml

In [10]:
model_path = "./"

In [11]:
model_name = 'final-model.pt'

In [12]:
model = torch.load(model_path + model_name)

In [13]:
# model = model.to_torchscript()
# torch.jit.save(script, "torchscript_model.pt")

In [14]:
# model.train()

In [16]:
bentoml.pytorch.save_model(
    "gat_lstm_autoencoder_reconstruction_loss:v1", 
    model
)

Model(tag="gat_lstm_autoencoder_reconstruction_loss:v1", path="/home/bentoml/bentoml/models/gat_lstm_autoencoder_reconstruction_loss/v1/")

In [17]:
!bentoml models list

 Tag                             Module           Size      Creation Time       
 gat_lstm_autoencoder_reconstr…  bentoml.pytorch  3.18 MiB  2023-06-13 09:10:24 


In [18]:
!bentoml models get gat_lstm_autoencoder_reconstruction_loss:v1

name: gat_lstm_autoencoder_reconstruction_loss
version: v1
module: bentoml.pytorch
labels: {}
options:
  partial_kwargs: {}
metadata: {}
context:
  framework_name: torch
  framework_versions:
    torch: 2.0.1
  bentoml_version: 1.0.18
  python_version: 3.9.13
signatures:
  __call__:
    batchable: false
api_version: v1
creation_time: '2023-06-13T09:10:24.272416+00:00'



In [19]:
loaded_model = bentoml.pytorch.load_model("gat_lstm_autoencoder_reconstruction_loss:v1")

In [19]:
data = torch.Tensor([[[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]]])

In [20]:
data.shape

torch.Size([24, 12, 20])

In [20]:
print(loaded_model.eval())
# output = loaded_model(data)
output = loaded_model(torch.ones([24,12,22]))

gat_lstm_autoencoder(
  (gat): GATLayer(
    (projection): Linear(in_features=22, out_features=16, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
  )
  (gat1): GATLayer(
    (projection): Linear(in_features=16, out_features=12, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
  )
  (lstm): LSTM(144, 144, num_layers=2, dropout=0.25)
  (lstm1): LSTM(144, 144, num_layers=2, dropout=0.25)
  (gat3): GATLayer(
    (projection): Linear(in_features=12, out_features=16, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
  )
  (gat4): GATLayer(
    (projection): Linear(in_features=16, out_features=24, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.2)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=288, out_features=264, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (linear): Linear(in_features=288, out_features=264, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (dropout1): Dropout(p=0.3, inplace=False)
  (dropout2): Dropout(p=0.3

/opt/conda/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [21]:
type(output)

torch.Tensor

In [22]:
output.shape

torch.Size([24, 1])

In [23]:
output

tensor([[0.8821],
        [0.8644],
        [0.8543],
        [0.8490],
        [0.8478],
        [0.8487],
        [0.8503],
        [0.8520],
        [0.8536],
        [0.8547],
        [0.8556],
        [0.8562],
        [0.8567],
        [0.8570],
        [0.8572],
        [0.8574],
        [0.8575],
        [0.8576],
        [0.8576],
        [0.8576],
        [0.8577],
        [0.8577],
        [0.8577],
        [0.8577]], grad_fn=<ReshapeAliasBackward0>)

In [25]:
# print(len(output), "x", len(output[0]), "x", len(output[0][0]), "x", len(output[0][0][0]))
# for a in output:
#     print([len(i) for i in a])

In [24]:
output.detach().numpy()

array([[0.8821251 ],
       [0.86443216],
       [0.85430044],
       [0.84900826],
       [0.8478122 ],
       [0.848668  ],
       [0.85030645],
       [0.85204625],
       [0.85356027],
       [0.85474384],
       [0.8556163 ],
       [0.8562405 ],
       [0.8566813 ],
       [0.85699147],
       [0.8572096 ],
       [0.85736364],
       [0.85747284],
       [0.85755056],
       [0.8576061 ],
       [0.857646  ],
       [0.8576748 ],
       [0.8576956 ],
       [0.85771066],
       [0.8577216 ]], dtype=float32)

Save the `clf` model instance to BentoML local model store:

Models saved can be accessed via `bentoml models` CLI command:

To verify that the saved model can be loaded correctly:

In BentoML, the recommended way of running ML model inference in serving is via Runner, which 
gives BentoML more flexibility in terms of how to schedule the inference computation, how to 
batch inference requests and take advantage of hardware resources available. Saved models can
be loaded as a Runner instance as shown below:


In [27]:
# # Create a Runner instance:
# model_runner = bentoml.pytorch.get("gat_lstm_autoencoder:latest").to_runner()

# # Runner#init_local initializes the model in current process, this is meant for development and testing only:
# model_runner.init_local()

# # This should yield the same result as the loaded model:
# model_runner.run(data, adj_matrix)

## Serving the model

A simple BentoML Service that serves the model saved above looks like this:

In [25]:
%%writefile service.py
import numpy as np
import bentoml
import torch
from bentoml.io import NumpyNdarray

model_runner = bentoml.pytorch.load_model("gat_lstm_autoencoder_reconstruction_loss:v1").eval()

galf = bentoml.Service("gat_lstm_autoencoder_reconstruction_loss_classifier")

print(dir(model_runner))
adj_matrix = torch.Tensor([#['payment 0', 'shipping 1', 'redis 2', 'mongodb 3', 'dispatch 4', 'rabbitmq 5', 'user 6', 'mysql 7', 'catalogue 8', 'ratings 9', 'web 10', 'cart 11']
       [[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]]])

# data = torch.Tensor([[[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]], [[45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1], [45056.0, 22585344.0, 27111424.0, 27295744.0, 0.00999999999999995, 0.001054579, 0.0700000000000001, 539.0, 0.0, 0.0, 4.0, 2516.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1]]])
@galf.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
#     print("###########################################")
#     print("###########################################")
#     print(input_series.shape)
#     print("###########################################")
#     print("###########################################")
    out = model_runner(torch.from_numpy(input_series).to(torch.float32)).detach().numpy()
    print(out.shape)
    return out


Overwriting service.py


In [26]:
data.dtype

NameError: name 'data' is not defined

Note: using `%%writefile` here because `bentoml.Service` definition must be created in its own `.py` file

Start a dev model server to test out the service defined above:

In [ ]:
!bentoml serve service.py:galf --reload

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_


Open your web browser at http://127.0.0.1:3000 to view the Bento UI for sending test requests.

You may also send request with `curl` command or any HTTP client, e.g.:

```bash
curl -X POST -H "content-type: application/json" --data "[[5.9, 3, 5.1, 1.8]]" http://127.0.0.1:3000/classify
```


### Build Bento for deployment

Bento is the distribution format in BentoML which captures all the source code, model files, config
files and dependency specifications required for running the service for production deployment. Think 
of it as Docker/Container designed for machine learning models.

To begin with building Bento, create a `bentofile.yaml` under your project directory:

In [ ]:
%%writefile bentofile.yaml
service: "service.py:galf"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
python:
  packages:
    - scikit-learn
    - pandas
    - torch
    - numpy
    - seaborn
    - matplotlib

Next, run `bentoml build` from current directory to start the Bento build:

In [ ]:
!bentoml build

A new Bento is now built and saved to local Bento store. You can view and manage it via 
`bentoml list`,`bentoml get` and `bentoml delete` CLI command.

## Containerize and Deployment

Bento is designed to be deployed to run efficiently in a variety of different environments.
And there are lots of deployment options and tools as part of the BentoML eco-system, such as 
[Yatai](https://github.com/bentoml/Yatai) and [bentoctl](https://github.com/bentoml/bentoctl) for
direct deployment to cloud platforms.

In this guide, we will show you the most basic way of deploying a Bento, which is converting a Bento
into a Docker image containing the HTTP model server.

Make sure you have docker installed and docker deamon running, and run the following commnand:

```bash
bentoml containerize iris_classifier:latest
```

This will build a new docker image with all source code, model files and dependencies in place,
and ready for production deployment. To start a container with this docker image locally, run:

```bash
docker run -p 3000:3000 iris_classifier:invwzzsw7li6zckb2ie5eubhd 
```

## What's Next?

- 👉 [Pop into our Slack community!](https://l.linklyhq.com/l/ktO8) We're happy to help with any issue you face or even just to meet you and hear what you're working on.

- Dive deeper into the [Core Concepts](https://docs.bentoml.org/en/latest/concepts/index.html) in BentoML
- Learn how to use BentoML with other ML Frameworks at [Frameworks Guide](https://docs.bentoml.org/en/latest/frameworks/index.html) or check out other [gallery projects](https://github.com/bentoml/BentoML/tree/main/examples)
- Learn more about model deployment options for Bento:
  - [🦄️ Yatai](https://github.com/bentoml/Yatai): Model Deployment at scale on Kubernetes
  - [🚀 bentoctl](https://github.com/bentoml/bentoctl): Fast model deployment on any cloud platform
